# ITRC Population and economic scenarios

This notebook extracts and reformats ITRC population and economic scenarios from NISMOD-DB

In [ ]:
import csv
import os

import pandas as pd
import requests

## Acquire datasets

Copy tables from NISMOD-DB to csv - the following cell prints SQL commands to run on the ITRC server.

These have been extracted and placed on the NISMOD FTP server at `/data/economics/population-economic-tables-nismod-db.zip`

In [ ]:
path = "D:\Data"
tables = [
    'population_scenarios',
    'population_scenario_combination_definitions',
    'Population_By_GOR',
    'Population_By_Districts',
    'Household_By_GOR',
    'Household_By_Districts',
    'economics_scenarios',
    'fossil_fuel_price_scenarios',
    'world_economic_growth_scenarios',
    'uk_sectors',
    'regional_sectors',
    'fossil_fuel_price_scenarios',
    'fossil_fuel_price_scenarios_values',
    'wholesale_fuel_price_fuels',
    'world_economic_growth_scenarios',
    'world_economic_growth_scenarios_values',
    'economics_economicsuknetoutputgvabyindustry',
    'government_office_regions',
    'economics_economicsuknetoutputgvabyindustryandregion'
]
for table in tables:
    print("COPY (SELECT * FROM \"{table}\") TO '{path}\{table}.txt' WITH CSV HEADER;".format(
        table=table,
        path=path))

In [ ]:
def download(url, file_path):
    """Download a file from a URL to a given path
    """
    if os.path.exists(file_path):
        print("Already have", file_path)
    else:
        response = requests.get(url)
        with open(file_path, 'wb') as fd:
            for chunk in response.iter_content(chunk_size=1024):
                fd.write(chunk)
        print("Downloaded", file_path)

Get ONS Local Authority Districts (December 2016) Names and Codes in the United Kingdom

In [ ]:
download(
    'https://opendata.arcgis.com/datasets/464be6191a434a91a5fa2f52c7433333_0.csv',
    os.path.join("raw_data", "lad_names_codes.csv")
)

Download Northern Ireland population mid-year estimates.

In [ ]:
download(
    "https://www.nisra.gov.uk/sites/nisra.gov.uk/files/publications/LGD14-tots-0116.xls",
    os.path.join("raw_data", "LGD14-tots-0116.xls")
)

Local Authority District to Region (December 2016) Lookup in England

In [ ]:
download(
    'https://opendata.arcgis.com/datasets/46d0cf00f75e4e4d8f5703c1cee283da_0.csv',
    os.path.join("raw_data", "lad_to_region.csv")
)

## Population

Population scenarios are generated using three scenario elements: high or low prosperity, sustainability and isolation, which correspond to strength and multiplier values for fertility, mortality and migration.

Scenarios are available in two spatial resolutions: Government Office Regions (GOR) and Local Authority Districts (LAD)

In [ ]:
a = pd.read_csv('raw_data/population_scenarios.txt').sort_values(by='id')
b = pd.read_csv('raw_data/population_scenario_combination_definitions.txt').sort_values(by='id').drop('scenario', axis=1)
population_scenario_definitions = pd.merge(a, b, on='id', validate='one_to_one').drop('id_', axis=1)
population_scenario_definitions

### Population by GOR

In [ ]:
population_gor = pd.read_csv(
    'raw_data/Population_By_GOR.txt'
).sort_values(
    by=['population_scenario_id', 'gor_id', 'gender', 'year']
).drop(
    ['gender', 'category', 'gor_id', 'id', 'population_scenario_id'], axis=1
).rename(
    columns={'location': 'government_office_region'}
)
population_gor

In [ ]:
population_gor.population_scenario.value_counts()

### Population by LAD

In [ ]:
population_lad = pd.read_csv(
    'raw_data/Population_By_Districts.txt'
).sort_values(
    by=['population_scenario_id', 'year', 'location']
).drop(
    ['gender', 'category', 'district_id', 'id', 'population_scenario_id'], axis=1
).dropna()
population_lad

In [ ]:
# fix Aberdeen City value
population_lad.loc[522343, 'population_scenario']  = "A"

In [ ]:
len(population_lad.location.unique())

In [ ]:
population_lad.population_scenario.value_counts()

### Match districts to 2016 LADs

In [ ]:
population_lad.location = population_lad.location.apply(lambda l: str(l).replace(' UA', ''))

Disaggregate Northern Ireland

In [ ]:
nire = population_lad[population_lad.location.isin(['NIRE', 'Northern Ireland'])].copy()
nire.location = 'Northern Ireland'
nire

In [ ]:
ni_pop = pd.read_excel(
    "raw_data/LGD14-tots-0116.xls", index_col=[0,1], header=[1]
).dropna()[0:12].reset_index().melt(
    id_vars=['level_0', 'level_1'],
    var_name='year',
    value_name='population'
).rename(
    columns={
        'level_0': 'lad_name',
        'level_1': 'lad_code'
    }
).query(
    'year == 2016'  # Pick base year for disaggregation
)
ni_total = ni_pop[ni_pop.lad_code == 'N92000002']
ni_lad = ni_pop[ni_pop.lad_code != 'N92000002'].copy()
ni_lad['location'] = 'Northern Ireland'

In [ ]:
ni_total

In [ ]:
ni_lad

In [ ]:
base_total = ni_total.population.values[0]
base_total

In [ ]:
nire_disagg = pd.merge(
    nire, ni_lad,
    on='location' 
).drop(
    columns=['year_y']
).rename(
    columns={
        'year_x': 'year',
        'population': 'base_population',
        'value': 'scenario_total'
    }
)
nire_disagg['value'] = nire_disagg.base_population * nire_disagg.scenario_total / base_total
nire_disagg = nire_disagg.drop(
    columns=['scenario_total', 'base_population', 'location', 'lad_name']
).rename(
    columns={'lad_code': 'location'}
)
nire_disagg

In [ ]:
population_lad = pd.concat([
    population_lad[~population_lad.location.isin(['NIRE', 'Northern Ireland'])],
    nire_disagg
])
population_lad

Apply name fixes and assumptions about LADs merging post-2009

In [ ]:
rename = {
    'Basingstoke and Dean': 'Basingstoke and Deane',
    'Bath and North East Some': 'Bath and North East Somerset',
    'Berwick-upon-Tweed': 'Berwick upon Tweed',
    'County of Herefordshire': 'Herefordshire, County of',
    'Hinckley and Boswort': 'Hinckley and Bosworth',
    "King's Lynn and West": "King's Lynn and West Norfolk",
    'Kingston upon Hull, City': 'Kingston upon Hull, City of',
    'North East Derbyshir': 'North East Derbyshire',
    'North West Leicester': 'North West Leicestershire',
    'Nuneaton and Bedwort': 'Nuneaton and Bedworth',
    'Rhondda, Cynon, Taff': 'Rhondda Cynon Taf',
    'South Northamptonshi': 'South Northamptonshire',
    'St. Albans': 'St Albans',
    'St. Edmundsbury': 'St Edmundsbury',
    'Staffordshire Moorla': 'Staffordshire Moorlands',
    'Stratford on Avon': 'Stratford-on-Avon',
    'Tonbridge and Mallin': 'Tonbridge and Malling',
    'The Vale of Glamorgan': 'Vale of Glamorgan',
    'Weymouth and Portlan': 'Weymouth and Portland',

    'Shrewsbury and Atcha': 'Shrewsbury and Atcham',
    'Ellesmere Port & Nes': 'Ellesmere Port & Neston',

    # rename (Western Isles in English)
    'Na H-Eileanan an Iar': 'Na h-Eileanan Siar',

    # Scilly Isles
    '15UH': '00HF',

    # Old code, looks like Suffolk because of the '42'
    # BUT is close to Surrey Heath (43UJ) which had zeroes otherwise
    '42UJ': '43UJ',
    
    # old codes to names
    '09UC': 'Mid Bedfordshire',
    '09UE': 'South Bedfordshire',
    '13UB': 'Chester',
    '13UC': 'Congleton',
    '13UD': 'Crewe and Nantwich',
    '13UE': 'Ellesmere Port & Neston',
    '13UG': 'Macclesfield',
    '13UH': 'Vale Royal',
    '15UB': 'Caradon',
    '15UC': 'Carrick',
    '15UD': 'Kerrier',
    '15UE': 'North Cornwall',
    '15UF': 'Penwith',
    '15UG': 'Restormel',
    '20UB': 'Chester-le-Street',
    '20UD': 'Derwentside',
    '20UE': 'Durham',
    '20UF': 'Easington',
    '20UG': 'Sedgefield',
    '20UH': 'Teesdale',
    '20UJ': 'Wear Valley',
    '35UB': 'Alnwick',
    '35UC': 'Berwick upon Tweed',
    '35UD': 'Blyth Valley',
    '35UE': 'Castle Morpeth',
    '35UF': 'Tynedale',
    '35UG': 'Wansbeck',
    '39UB': 'Bridgnorth',
    '39UC': 'North Shropshire',
    '39UD': 'Oswestry',
    '39UE': 'Shrewsbury and Atcham',
    '39UF': 'South Shropshire',
    '46UB': 'Kennet',
    '46UC': 'North Wiltshire',
    '46UD': 'Salisbury',
    '46UF': 'West Wiltshire'
}

# See e.g. https://en.wikipedia.org/wiki/2009_structural_changes_to_local_government_in_England
merge = {
    # Two-tier structures Merged to single authority
    'Caradon': 'Cornwall',
    'Carrick': 'Cornwall',
    'Kerrier': 'Cornwall',
    'North Cornwall': 'Cornwall',
    'Penwith': 'Cornwall',
    'Restormel': 'Cornwall',

    'Alnwick': 'Northumberland',
    'Berwick upon Tweed': 'Northumberland',
    'Blyth Valley': 'Northumberland',
    'Castle Morpeth': 'Northumberland',
    'Tynedale': 'Northumberland',
    'Wansbeck': 'Northumberland',
    'Wear Valley': 'Northumberland',

    'Bridgnorth': 'Shropshire',
    'North Shropshire': 'Shropshire',
    'Oswestry': 'Shropshire',
    'Shrewsbury and Atcham': 'Shropshire',
    'South Shropshire': 'Shropshire',

    'Chester-le-Street': 'County Durham',
    'Derwentside': 'County Durham',
    'Durham': 'County Durham',
    'Easington': 'County Durham',
    'Sedgefield': 'County Durham',
    'Teesdale': 'County Durham',
    'Wear Valley': 'County Durham',

    'Kennet': 'Wiltshire',
    'North Wiltshire': 'Wiltshire',
    'Salisbury': 'Wiltshire',
    'West Wiltshire': 'Wiltshire',

    # Merged into multiple authorities
    'Congleton': 'Cheshire East',
    'Crewe and Nantwich': 'Cheshire East',
    'Vale Royal': 'Cheshire East',

    'Chester': 'Cheshire West and Chester',
    'Ellesmere Port & Neston': 'Cheshire West and Chester',
    'Macclesfield': 'Cheshire West and Chester',

    'Mid Bedfordshire': 'Central Bedfordshire',
    'South Bedfordshire': 'Central Bedfordshire',

    # Bedford > Bedford
    '09UD': 'Bedford',
}

In [ ]:
old_to_new = {}
name_to_new = {}
with open(os.path.join('raw_data', 'lad_names_codes.csv'), 'r', encoding='utf-8-sig') as fh:
    r = csv.DictReader(fh)
    for line in r:
        old_to_new[line['LAD16CDO']] = line['LAD16CD']
        name_to_new[line['LAD16NM']] = line['LAD16CD']

In [ ]:
def lookup_location(location):
    if location in rename:
        location = rename[location]

    if location in merge:
        location = merge[location]   
        
    if location in name_to_new:
        location = name_to_new[location]
        
    if location in old_to_new:
        location = old_to_new[location]
        
    return location

In [ ]:
population_lad.location = population_lad.location.apply(lookup_location)

In [ ]:
sorted(list(population_lad.location.unique()))

In [ ]:
population_lad = population_lad.rename(
    columns={'location': 'lad_2016'}
)

### Households by GOR

In [ ]:
households_gor = pd.read_csv(
    'raw_data/Household_By_GOR.txt'
).sort_values(
    by=['population_scenario_id', 'gor_id', 'year']
).drop(
    ['population_scenario_id', 'gor_id', 'id'], axis=1
).rename(
    columns={'location': 'government_office_region'}
)
households_gor

### Households by LAD

In [ ]:
households_lad = pd.read_csv(
    'raw_data/Household_By_Districts.txt'
).sort_values(
    by=['population_scenario_id', 'district_id', 'year']
).drop(
    ['population_scenario_id', 'district_id', 'id'], axis=1
)

In [ ]:
households_lad.location = households_lad.location.apply(lambda l: str(l).replace(' UA', ''))
households_lad = households_lad[households_lad['location'] != 'NIRE']
households_lad = households_lad[households_lad['location'] != 'Northern Ireland']
households_lad.location = households_lad.location.apply(lookup_location)

In [ ]:
households_lad

## Economics

Economics scenarios are generated using three scenario elements: population, fossil fuel prices and world economic growth.

GVA is projected for a larger set of economic sectors for the whole of the UK, or with a smaller set of economic sectors for Government Office Regions (GOR). The sectors are mapped to the [UK Standard Industry Classification (SIC)](https://www.ons.gov.uk/methodology/classificationsandstandards/ukstandardindustrialclassificationofeconomicactivities/uksic2007), which is aligned with the Statistical Classification of Economic Activities in the European Community, Rev. 2 (2008) and the International Standard Industrial Classification of All Economic Activities, Rev. 4 (2008).

Population scenarios are time series for either LADs or GORs, as above. Fossil fuel price and world economic growth scenarios are both time series with global (or no) spatial resolution.

In [ ]:
econ_fuel = pd.merge(    
    pd.read_csv('raw_data/economics_scenarios.txt'), 
    pd.read_csv('raw_data/fossil_fuel_price_scenarios.txt'), 
    left_on='fossil_fuel_price_scenario_id', right_on='id')[[
    'id_x',
    'scenario_id',
    'scenario_description',
    'id_y',
    'code',
    'description',
    'population_scenario_id',
    'world_economic_growth_scenario_id'    
]].rename(
    columns={
        'id_x': 'scenario_id',
        'scenario_id': 'scenario_code',
        'scenario_description': 'scenario_desc',
        'id_y': 'fuel_s_id',
        'code': 'fuel_s_code',
        'description': 'fuel_s_desc',
    }
).sort_values(
    by='scenario_id'
)
econ_fuel.head()

In [ ]:
econ_fuel_growth = pd.merge(
    econ_fuel,
    pd.read_csv('raw_data/world_economic_growth_scenarios.txt'), 
    left_on='world_economic_growth_scenario_id', right_on='id')[[
    'scenario_id',
    'scenario_code',
    'scenario_desc',
    'population_scenario_id',
    'id',
    'code',
    'description',
    'fuel_s_id',
    'fuel_s_code',
    'fuel_s_desc',
]].rename(
    columns={
        'id': 'econ_s_id',
        'code': 'econ_s_code',
        'description': 'econ_s_desc',
    }
)
econ_fuel_growth

In [ ]:
econ_fuel_growth.population_scenario_id = econ_fuel_growth.population_scenario_id.apply(lambda d: d.strip())

In [ ]:
econ_fuel_growth.population_scenario_id.unique()

In [ ]:
population_scenario_definitions.scenario_id.unique()

In [ ]:
scenario_definitions = pd.merge(
    econ_fuel_growth, 
    population_scenario_definitions, 
    left_on='population_scenario_id', right_on='scenario_id', 
    validate='many_to_one'
)[[
    'scenario_id_x',
    'scenario_code',
    'scenario_desc',
    'econ_s_id',
    'econ_s_code',
    'econ_s_desc',
    'fuel_s_id',
    'fuel_s_code',
    'fuel_s_desc',
    'id', 
    'scenario_id_y',
    'population_scenario_id',
    'scenario_description',
    'base_census_year', 'prosperity', 'sustainability', 'isolation',
    'fertility_strength', 'mortality_strength', 'migration_strength',
    'fertility_multiplier', 'mortality_multiplier', 'migration_multiplier'
]].rename(
    columns={
        'scenario_id_x': 'scenario_id',
        'scenario_id_y': 'pop_s_id',
        'population_scenario_id': 'pop_s_code',
        'scenario_description': 'pop_s_desc'
    }
)
scenario_definitions

In [ ]:
scenario_definitions['scenario_code_new'] = scenario_definitions.apply(
    lambda r: "pop-{}_econ-{}_fuel-{}".format(r.pop_s_code, r.fuel_s_code, r.econ_s_code).lower(), axis=1
)
scenario_definitions['description']= scenario_definitions.apply(
    lambda r: """{} population scenario: {}.
{} world economic growth scenario.
{} fossil fuel price scenario.
""".format(
        r.pop_s_code, str(r.pop_s_desc).lower(), 
        r.fuel_s_code, 
        r.econ_s_code), axis=1
)

In [ ]:
scenario_definitions.description[0]

### Standard industrial classification

The UK Standard Industrial Classification(SIC) is aligned with the Statistical Classification of Economic Activities in the European Community, Rev. 2 (2008) and the International Standard Industrial Classification of All Economic Activities, Rev. 4 (2008).

These are all hierarchical classifications which use the same codes and descriptions - letters (A-T) at the top level, numbers (1-99) at the second level, and decimals for the third level (e.g. 6.1, 6.2).

The sectors used in the ITRC scenarios map closely to the SIC, with some grouping and splitting between levels, based on my reading of the sector names and SIC text descriptions.

In [ ]:
uk_sectors = pd.read_csv('raw_data/uk_sectors.txt')
uk_sectors

In [ ]:
regional_sectors = pd.read_csv('raw_data/regional_sectors.txt')
regional_sectors

## Fuel price scenarios

In [ ]:
a = pd.read_csv('raw_data/fossil_fuel_price_scenarios.txt')
b = pd.read_csv('raw_data/fossil_fuel_price_scenarios_values.txt')
c = pd.merge(a, b, left_on='id', right_on='scenario_id', validate='one_to_many')
d = pd.read_csv('raw_data/wholesale_fuel_price_fuels.txt')
fossil_fuel_price_scenarios = pd.merge(c, d, left_on='fuelid', right_on='wholesale_fuel_price_id', validate='many_to_one')[[
    'description',
    'code',
    'scenario_id',
    'year',
    'value',
    'wholesale_fuel_price_name',
    'wholesale_fuel_price_units'
]].sort_values(
    by=['code', 'year', 'wholesale_fuel_price_name']
).drop(
    ['description', 'scenario_id', 'wholesale_fuel_price_units'], axis=1
).rename(
    columns={'wholesale_fuel_price_name':'fuel'}
)
fossil_fuel_price_scenarios

In [ ]:
oil_price = fossil_fuel_price_scenarios[fossil_fuel_price_scenarios.fuel == 'Crude Oil'].drop('fuel', axis=1)
oil_price

In [ ]:
coal_price = fossil_fuel_price_scenarios[fossil_fuel_price_scenarios.fuel == 'Coal'].drop('fuel', axis=1)
coal_price

In [ ]:
gas_price = fossil_fuel_price_scenarios[fossil_fuel_price_scenarios.fuel == 'Gas'].drop('fuel', axis=1)
gas_price

## World economic growth scenarios

In [ ]:
a = pd.read_csv('raw_data/world_economic_growth_scenarios.txt')
b = pd.read_csv('raw_data/world_economic_growth_scenarios_values.txt')
world_economic_growth_scenarios = pd.merge(
    a, b, left_on='id', right_on='scenario_id', validate='one_to_many'
)[
    ['code', 'description', 'year', 'index_value', 'growth_value']
]
world_economic_growth_scenarios

## UK scenarios

In [ ]:
gva_uk_by_sector = pd.read_csv(
    'raw_data/economics_economicsuknetoutputgvabyindustry.txt'
).rename(
    columns={
        'file_uuid_id': 'scenario_id',
        'uk_industry_sector': 'economic_sector__uk'
    }
).drop(
    ['id', 'added_on'], axis=1
)
gva_uk_by_sector

## Regional scenarios

Government Office Regions are still used as NUTS1 statistical regions for the UK: Scotland, Wales, Northern Ireland and nine regions within England.

In [ ]:
regions = pd.read_csv('raw_data/government_office_regions.txt')[
    ['gid', 'name']
]
regions

In [ ]:
gva_gor_by_sector = pd.merge(
    pd.read_csv('raw_data/economics_economicsuknetoutputgvabyindustryandregion.txt'),
    regions,
    left_on='region', right_on='gid'
).rename(
    columns={
        'file_uuid_id': 'scenario_id',
        'region': 'region_id',
        'name': 'government_office_region',
        'regional_industry_sector': 'economic_sector__gor',
    }
).drop(
    ['added_on', 'id', 'gid', 'region_id'], axis=1
)

In [ ]:
gva_gor_by_sector

## GVA per capita

1. Calculate GVA per capita for regions (GORs)
1. Assign GVA per capita value to each LAD based on its parent GOR (no attempt to disaggregate or add information)

In [ ]:
gva_gor_with_pop_scenario = pd.merge(
    gva_gor_by_sector, 
    scenario_definitions[['scenario_id', 'pop_s_code']],
    on='scenario_id'
)
gva_gor_with_pop_scenario

In [ ]:
gva_pc_gor = pd.merge(
    gva_gor_with_pop_scenario.rename(columns={'value': 'gva'}), 
    population_gor.rename(
        columns={'value': 'population'}
    ),
    left_on=['pop_s_code', 'government_office_region', 'year'], 
    right_on=['population_scenario', 'government_office_region', 'year'],
    validate='many_to_one'
)
# [[
#     'population_scenario', 'year', 'government_office_region', 'economic_sector__gor', 'gva', 'population'
# ]]
gva_pc_gor['value'] = gva_pc_gor.gva * 1000000 / gva_pc_gor.population 
gva_pc_gor = gva_pc_gor.drop(['gva', 'population'], axis=1)

In [ ]:
gva_pc_gor

### Local Authority Districts

In [ ]:
def define_gor(row):
    country_letter = row.LAD16CD[0]
    if country_letter == 'E':
        return row.RGN16NM
    if country_letter == 'S':
        return 'Scotland'
    if country_letter == 'W':
        return 'Wales'
    if country_letter == 'N':
        return 'Northern Ireland'

In [ ]:
a = pd.read_csv(
    'raw_data/lad_to_region.csv'
).drop(
    ['LAD16NM', 'FID'], axis=1
)
print(len(a))

b = pd.read_csv(
    'raw_data/lad_names_codes.csv'
).drop(
    ['LAD16CDO', 'FID'], axis=1
)
print(len(b))

c = pd.merge(
    b, a, on='LAD16CD',
    how='left'
)
c.RGN16NM = c.apply(define_gor, axis=1)
print(len(c))

lad_gor = pd.merge(
    c, regions, 
    left_on='RGN16NM', right_on='name',
    how='left'
).drop(
    ['name','RGN16CD', 'LAD16NM', 'gid'], axis=1
).rename(
    columns={
        'LAD16CD': 'lad_2016',
        'RGN16CD': 'region_code',
        'RGN16NM': 'government_office_region',
        'LAD16NM': 'lad_2016_name'
    }
).sort_values(
    by='lad_2016'
)
print(len(lad_gor))

In [ ]:
lad_gor

# Output

Output all scenario data per economic scenario (which encompasses/relies on population, fuel prices, world growth).

In [ ]:
# how to handle components/parameters/definitions?
for s in scenario_definitions.itertuples():
    try:
        os.mkdir(os.path.join('data', s.scenario_code_new))
    except FileExistsError:
        pass

In [ ]:
# economic_sectors__regional (dimension, not data)
regional_sectors.rename(
    columns={
        'sector_id': 'id',
        'sector_name': 'description'
    }
).to_csv('data/economic_sectors__regional.csv', index=False)

In [ ]:
# economic_sectors__uk (dimension, not data)
uk_sectors.rename(
    columns={
        'sector_id': 'id',
        'sector_name': 'description'
    }
).to_csv('data/economic_sectors__uk.csv', index=False)

In [ ]:
# population__gor
for s in scenario_definitions.itertuples():
    fname = 'data/{}/population__gor.csv'.format(s.scenario_code_new)
    population_gor[
        population_gor.population_scenario == s.pop_s_code
    ][[
        'year',
        'government_office_region',
        'value'
    ]].rename(
        columns={'government_office_region': 'region'}
     ).to_csv(
        fname,
        index=False
    )

In [ ]:
# population__lad
for s in scenario_definitions.itertuples():
    fname = 'data/{}/population__lad.csv'.format(s.scenario_code_new)
    population_lad[
        population_lad.population_scenario == s.pop_s_code
    ][[
        'year',
        'lad_2016',
        'value'
    ]].rename(
        columns={'lad_2016': 'region'}
     ).sort_values(
        by=['year','region']
    ).to_csv(
        fname,
        index=False
    )

In [ ]:
# households__gor
for s in scenario_definitions.itertuples():
    fname = 'data/{}/households__gor.csv'.format(s.scenario_code_new)
    households_gor[
        households_gor.population_scenario == s.pop_s_code
    ][[
        'year',
        'government_office_region',
        'value'
    ]].rename(
        columns={'government_office_region': 'region'}
     ).to_csv(
        fname,
        index=False
    )

In [ ]:
# households__lad
for s in scenario_definitions.itertuples():
    fname = 'data/{}/households__lad.csv'.format(s.scenario_code_new)
    households_lad[
        households_lad.population_scenario == s.pop_s_code
    ][[
        'year',
        'location',
        'value'
    ]].rename(
        columns={'location': 'region'}
     ).sort_values(
        by=['year','region']
    ).to_csv(
        fname,
        index=False
    )

In [ ]:
# world_growth__index, world_growth__growth
for s in scenario_definitions.itertuples():
    i_fname = 'data/{}/world_growth__index.csv'.format(s.scenario_code_new) 
    g_fname = 'data/{}/world_growth__growth.csv'.format(s.scenario_code_new)
    
    world_economic_growth_scenarios.rename(
        columns={'index_value': 'value'}
    )[
        world_economic_growth_scenarios.code == s.econ_s_code
    ][
        ['year', 'value']
    ].to_csv(
        i_fname, index=False
    )
    
    world_economic_growth_scenarios.rename(
        columns={'growth_value': 'value'}
    )[
        world_economic_growth_scenarios.code == s.econ_s_code
    ][
        ['year', 'value']
    ].to_csv(
        g_fname, index=False
    )

In [ ]:
# coal_price, oil_price, gas_price
for s in scenario_definitions.itertuples():
    coal_price[
        coal_price.code == s.fuel_s_code
    ][
        ['year', 'value']
    ].to_csv(
        'data/{}/coal_price.csv'.format(s.scenario_code_new), index=False
    )  
    
    oil_price[
        oil_price.code == s.fuel_s_code
    ][
        ['year', 'value']
    ].to_csv(
        'data/{}/oil_price.csv'.format(s.scenario_code_new), index=False
    )  
    
    gas_price[
        gas_price.code == s.fuel_s_code
    ][
        ['year', 'value']
    ].to_csv(
        'data/{}/gas_price.csv'.format(s.scenario_code_new), index=False
    )

In [ ]:
# gva__uk
for s in scenario_definitions.itertuples():
    gva_uk_by_sector[
        gva_uk_by_sector.scenario_id == s.scenario_id
    ][
        ['year', 'value']
    ].groupby('year').sum().to_csv('data/{}/gva__uk.csv'.format(s.scenario_code_new))

In [ ]:
# gva__uk_sector
for s in scenario_definitions.itertuples():
    gva_uk_by_sector[
        gva_uk_by_sector.scenario_id == s.scenario_id
    ][
        ['economic_sector__uk', 'year', 'value']
    ].to_csv('data/{}/gva__uk_sector.csv'.format(s.scenario_code_new), index=False)

In [ ]:
# gva__gor_sector
for s in scenario_definitions.itertuples():
    gva_gor_by_sector[
        gva_gor_by_sector.scenario_id == s.scenario_id
    ][
        ['government_office_region', 'economic_sector__gor', 'year', 'value']
    ].to_csv('data/{}/gva__gor_sector.csv'.format(s.scenario_code_new), index=False)

In [ ]:
# gva__gor
for s in scenario_definitions.itertuples():
    gva_gor_by_sector[
        gva_gor_by_sector.scenario_id == s.scenario_id
    ][
        ['government_office_region', 'year', 'value']
    ].groupby(['government_office_region', 'year']).sum().reset_index().to_csv(
        'data/{}/gva__gor.csv'.format(s.scenario_code_new), index=False
    )

In [ ]:
# gva_per_head__gor_sector
for s in scenario_definitions.itertuples():
    gva_pc_gor[
        gva_pc_gor.scenario_id == s.scenario_id
    ][
        ['government_office_region', 'economic_sector__gor', 'year', 'value']
    ].to_csv(
        'data/{}/gva_per_head__gor_sector.csv'.format(s.scenario_code_new), index=False
    )

In [ ]:
# gva_per_head_gor
for s in scenario_definitions.itertuples():
    gva_pc_gor[
        gva_pc_gor.scenario_id == s.scenario_id
    ].drop(
        ['scenario_id', 'economic_sector__gor'], axis=1
    ).groupby(
        ['government_office_region', 'year']
    ).sum().reset_index().to_csv(
        'data/{}/gva_per_head__gor.csv'.format(s.scenario_code_new), index=False
    )

In [ ]:
gva_pc_gor

In [ ]:
# gva_per_head__lad
for s in scenario_definitions.itertuples():
    pd.merge(
        lad_gor,
        gva_pc_gor[
            gva_pc_gor.scenario_id == s.scenario_id
        ].drop(
            ['economic_sector__gor', 'pop_s_code', 'population_scenario'], axis=1
        ).groupby(
            ['scenario_id', 'government_office_region', 'year']
        ).sum().reset_index(),
        on='government_office_region',
        how='left'
    ).drop(
        ['government_office_region', 'scenario_id'], axis=1
    ).rename(
        columns={'lad_2016': 'region'}
    ).to_csv(
        'data/{}/gva_per_head__lad.csv'.format(s.scenario_code_new), index=False
    )

In [ ]:
# gva_per_head__lad_sector
for s in scenario_definitions.itertuples():
    pd.merge(
        lad_gor,
        gva_pc_gor[
            gva_pc_gor.scenario_id == s.scenario_id
        ].drop(
            ['pop_s_code', 'population_scenario'], axis=1
        ),
        on='government_office_region',
        how='left'
    ).drop(
        ['government_office_region', 'scenario_id'], axis=1
    ).rename(
        columns={'lad_2016': 'region'}
    ).to_csv(
        'data/{}/gva_per_head__lad_sector.csv'.format(s.scenario_code_new), index=False
    )